In [1]:
import numpy as np
import sys
import CRPS.CRPS as pscore

sys.path.append('../../Evaluation/')
from conduct_evaluation import ConductEvaluation

from normal_evaluation.gamlss_evaluation import SampleOutcomes_GAMLSS_LogNormal

import pandas as pd

In [2]:
sample_model = SampleOutcomes_GAMLSS_LogNormal
trained_model = None
sample_model_kwargs = {}
N = 1000
n_processes = 50

get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

Evaluation index based

In [3]:
train_event_log = pd.read_csv("../../../data/PCR/predictions_logno_indexbased/PCR_test.csv", parse_dates=["time:timestamp_start", "time:timestamp_complete"])
test_event_log = pd.read_csv("../../../data/PCR/predictions_logno_indexbased/PCR_train.csv", parse_dates=["time:timestamp_start", "time:timestamp_complete"])

for col in ["time:timestamp_start", "time:timestamp_complete"]:
    train_event_log[col] = pd.to_datetime(train_event_log[col], errors="coerce")
    test_event_log[col] = pd.to_datetime(test_event_log[col], errors="coerce")

evaluator_train = ConductEvaluation(
    trained_model=trained_model,
    sample_model_type=sample_model,
    sample_model_kwargs=sample_model_kwargs,
    event_log=train_event_log,
    n=N,
    n_processes=n_processes
)

evaluator_test = ConductEvaluation(
    trained_model=trained_model,
    sample_model_type=sample_model,
    sample_model_kwargs=sample_model_kwargs,
    event_log=test_event_log,
    n=N,
    n_processes=n_processes
)

likelihoods_test = evaluator_test.sample_cases(False, True)
likelihoods_train = evaluator_train.sample_cases(False, True)

ll_test = np.mean([v.ln() for v in likelihoods_test[0].values()])
ll_train = np.mean([v.ln() for v in likelihoods_train[0].values()])
crsp_train = np.mean(get_pscores(likelihoods_train))
crsp_test = np.mean(get_pscores(likelihoods_test))


100%|██████████| 1202/1202 [00:57<00:00, 21.05it/s]


In [4]:
results = {
    "ll_test": np.mean([v.ln() for v in likelihoods_test[0].values()]),
    "ll_train": np.mean([v.ln() for v in likelihoods_train[0].values()]),
    "crsp_train": np.mean(get_pscores(likelihoods_train)),
    "crsp_test": np.mean(get_pscores(likelihoods_test))
}
results

{'ll_test': Decimal('-7.776294974065334768241492304'),
 'll_train': Decimal('-7.967873792858408851024523017'),
 'crsp_train': np.float64(662091768104202.4),
 'crsp_test': np.float64(3107584026268855.0)}